# Main Script to Load, Transform and Analyse data for Final Assigment

## Data load from local file "HM Land Registry - Price Paid Data property sales in Greater London - Raw Data.csv"

# Import required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml # Used to manage configuration Parameters, I'm using it at work and it's so easy to set configurable constant values 

# Loading config file

In [2]:
cnf_file = open(f"config.yaml", 'r')
conf_dictionary = yaml.full_load(cnf_file)
config = conf_dictionary["default"]

# Load data from CSV file

In [3]:
df1 = pd.read_csv("Property sales in Greater London - 2018 - 2019.csv", parse_dates=[config['parse_dates']])
df2 = pd.read_csv("Property sales in Greater London - 2020 - 2023.csv", parse_dates=[config['parse_dates']])

# Merging both dataframes

In [4]:
df = pd.concat([df1, df2])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 489352 entries, 0 to 287274
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   unique_id             489352 non-null  object        
 1   price_paid            489352 non-null  int64         
 2   deed_date             489352 non-null  datetime64[ns]
 3   postcode              488916 non-null  object        
 4   property_type         489352 non-null  object        
 5   new_build             489352 non-null  object        
 6   estate_type           489352 non-null  object        
 7   saon                  183047 non-null  object        
 8   paon                  489352 non-null  object        
 9   street                489312 non-null  object        
 10  locality              31256 non-null   object        
 11  town                  489352 non-null  object        
 12  district              489352 non-null  object        
 13 

# Renaming columns from both DFs

keys   = ['unique_id','price_paid','deed_date','postcode','property_type','new_build','estate_type','saon','paon','street','locality','town','district','county','transaction_category','linked_data_uri']
values = ['Id','Value','Purchased On','Postcode','Property Type','New Build','Estate Type','Saon','Paon','Street','Locality','Town','District','County','Transaction Category','URL']
dict_cols = dict(zip(keys, values))
df.rename(columns=dict_cols, inplace=True)
col_order = ['Id','Purchased On','Postcode','Property Type','Value','New Build','Estate Type','Transaction Category','Saon','Paon','Street','Locality','Town','District','County','URL']
df = df[col_order]

# Explore the data

In [5]:
print("Dataframe head:")
print(df.head())

print("Dataframe shape:")
print(df1.shape)
print(df2.shape)
print(df.shape )

print("Dataframe columns:")
print(df.columns)


Dataframe head:
                              unique_id  price_paid  deed_date postcode  \
0  7E86B6FB-423C-458C-E053-6B04A8C0C84C       99000 2018-11-30  BR1 1AA   
1  9DBAD222-2B0C-6EB3-E053-6B04A8C0F257      445000 2019-12-09  BR1 1AB   
2  965B6D91-D514-95E4-E053-6C04A8C07729      255000 2019-10-16  BR1 1AE   
3  7E86B6FB-4232-458C-E053-6B04A8C0C84C      400000 2018-11-23  BR1 1AF   
4  9FF0D96A-1CCA-11ED-E053-6C04A8C06383      380000 2019-07-05  BR1 1AF   

  property_type new_build estate_type         saon                     paon  \
0             F         N           L      FLAT 28              HENRY HOUSE   
1             F         N           L  APARTMENT 1  THE OLD POST OFFICE, 3B   
2             F         N           L       FLAT 2         HAWKSWORTH HOUSE   
3             F         N           L      FLAT 12            WILLIAM HOUSE   
4             F         N           L      FLAT 14            WILLIAM HOUSE   

         street locality     town district          county

# Index of Property Market - Yearly Variation